In [25]:
from chess_evaluator import evaluate_board, piece_values
import chess
import random

In [26]:
def score_move(board, move):
    """
    Scores moves based on heuristics to improve move ordering.
    Higher scores indicate better moves.
    """
    score = 0

    # Prioritize captures using MVV-LVA (Most Valuable Victim - Least Valuable Attacker)
    if board.is_capture(move):
        captured_piece = board.piece_at(move.to_square)
        if captured_piece:
            attacker = board.piece_at(move.from_square)
            score += 10 * piece_values[captured_piece.piece_type] - piece_values[attacker.piece_type]

    # Prioritize promotions
    if move.promotion:
        score += 800  # High score for promotions

    # Prioritize checks
    board.push(move)
    if board.is_check():
        score += 50
    board.pop()

    # Penalize moves that hang a piece
    board.push(move)
    if board.is_attacked_by(not board.turn, move.to_square):
        if not board.is_attacked_by(board.turn, move.to_square):
            # The piece is hanging
            score -= piece_values[board.piece_type_at(move.to_square)]
    board.pop()

    return score


In [27]:
killer_moves = {}

def minimax(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or board.is_game_over():
        return evaluate_board(board)

    global killer_moves
    killer_move = killer_moves.get(depth)

    if maximizing_player:
        max_eval = -float('inf')
        moves = list(board.legal_moves)

        # Place the killer move at the front if it exists
        if killer_move in moves:
            moves.remove(killer_move)
            moves.insert(0, killer_move)

        moves.sort(key=lambda move: score_move(board, move), reverse=True)

        for move in moves:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, False)
            board.pop()
            if eval > max_eval:
                max_eval = eval
                if eval > alpha:
                    alpha = eval
                    # Update killer move
                    killer_moves[depth] = move
            if beta <= alpha:
                break  # Beta cutoff
        return max_eval
    else:
        min_eval = float('inf')
        moves = list(board.legal_moves)

        # Place the killer move at the front if it exists
        if killer_move in moves:
            moves.remove(killer_move)
            moves.insert(0, killer_move)

        moves.sort(key=lambda move: score_move(board, move), reverse=True)

        for move in moves:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, True)
            board.pop()
            if eval < min_eval:
                min_eval = eval
                if eval < beta:
                    beta = eval
                    # Update killer move
                    killer_moves[depth] = move
            if beta <= alpha:
                break  # Alpha cutoff
        return min_eval


In [28]:
def select_best_move(board, depth):
    best_move = None
    max_eval = -float('inf')
    alpha = -float('inf')
    beta = float('inf')

    moves = list(board.legal_moves)
    moves.sort(key=lambda move: score_move(board, move), reverse=True)

    for move in moves:
        board.push(move)
        eval = minimax(board, depth - 1, alpha, beta, False)
        board.pop()
        if eval > max_eval:
            max_eval = eval
            best_move = move
        alpha = max(alpha, eval)
    return best_move

In [29]:
def play_game():
    board = chess.Board()
    depth = 3  # Set the desired depth

    while not board.is_game_over():
        if board.turn == chess.WHITE:
            # Bot's move
            move = select_best_move(board, depth)
        else:
            # For testing, opponent makes a random move
            move = random.choice(list(board.legal_moves))
        board.push(move)
        print(board)
        print(f"Evaluation: {evaluate_board(board)}\n")


In [30]:
play_game()

AttributeError: module 'chess' has no attribute 'square_ring'